# Dateset utils function

In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import seaborn as sn
import numpy as np
from scipy import ndimage

import random
import PIL
import os
import cv2 as cv2
import itertools
import glob

In [2]:
def loadImages(path, labels, dimension=64): #functio to retrive the sets
    y = []
    x = []
    
    for index,name in enumerate(labels):
        s = path+'/'+name+'/{}'
        temp = [s.format(i) for i in os.listdir(path+'/'+name+'/')]
        for image in temp:
            x.append(cv2.resize(cv2.imread(image,cv2.IMREAD_GRAYSCALE),(dimension,dimension),interpolation=cv2.INTER_CUBIC))
            y.append(index)
             
    x = np.asarray([np.reshape(im, (dimension, dimension, 1)) for im in x])
    x = x/255 #perchè è scala di grigi? per normalizzare cosa?
    y = np.asarray(y)
    return x, y

In [3]:
def dataAugmentation(path, labels, dimension=64):
    x = [] #New ImageSet
    y = [] #New Label
    
    for index,name in enumerate(labels):
        s = path+'/'+name+'/{}'
        temp = [s.format(i) for i in os.listdir(path+'/'+name+'/')]
        for image in temp:
            tmp = cv2.resize(cv2.imread(image,cv2.IMREAD_GRAYSCALE),(dimension,dimension),interpolation=cv2.INTER_CUBIC)
            x.append(tmp)
            x.append(cv2.flip(tmp,1))
            y.append(index)
            y.append(index)
        
    x = np.asarray([np.reshape(im, (64, 64, 1)) for im in x])
    x=x/255
    y = np.asarray(y)
    return x,y

In [4]:
def predictClass(prediction):
    y = [] 
    for x in prediction: 
        y.append(np.argmax(x))
    return y

In [5]:
def get_random_crop(img, width, height, dim=64):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    resImg = cv2.resize(img, (dim,dim), interpolation=cv2.INTER_CUBIC)
    return resImg

In [6]:
import math
import copy

def warpAffine(src, M, dsize, from_bounding_box_only=False):
    """
    Applies cv2 warpAffine, marking transparency if bounding box only
    The last of the 4 channels is merely a marker. It does not specify opacity in the usual way.
    """
    return cv2.warpAffine(src, M, dsize)

def rotate_image(image, dim=64):
    """Rotate the image counterclockwise.
    Rotate the image such that the rotated image is enclosed inside the
    tightest rectangle. The area not occupied by the pixels of the original
    image is colored black.
    Parameters
    ----------
    image : numpy.ndarray
        numpy image
    angle : float
        angle by which the image is to be rotated. Positive angle is
        counterclockwise.
    Returns
    -------
    numpy.ndarray
        Rotated Image
    """
    angle = random.randint(0, 360)
    # get dims, find center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = warpAffine(image, M, (nW, nH), False)

    resImg = cv2.resize(image, (dim,dim), interpolation=cv2.INTER_CUBIC)

    return resImg

In [10]:
def dataAugmentationWithCropRot(path, labels, dimension=64, d=1):
    x = [] #New ImageSet
    y = [] #New Label
    
    for index,name in enumerate(labels):
        s = path+'/'+name+'/{}'
        temp = [s.format(i) for i in os.listdir(path+'/'+name+'/')]
        for image in temp:
            tmp = cv2.resize(cv2.imread(image,cv2.IMREAD_GRAYSCALE),(dimension,dimension),interpolation=cv2.INTER_CUBIC)
            x.append(tmp)
            x.append(cv2.flip(tmp,1))
            x.append(get_random_crop(tmp, dimension-24, dimension-24, dimension))
            x.append(rotate_image(tmp, dimension))
            y.append(index)
            y.append(index)
            y.append(index)
            y.append(index)
        
    x = np.asarray([np.reshape(im, (dimension, dimension, d)) for im in x])
    x=x/255
    y = np.asarray(y)
    return x,y